# Emissions Compliance Forecasting Model

## Overview
This notebook develops an AI/ML module to forecast emission rates (SO2, NOx, CO2) for power plant operations to ensure compliance with EPA 40 CFR Part 75. The module is designed for Rayfield Systems' energy compliance software to automate emissions monitoring and prevent regulatory violations.

## Problem Definition

**Energy Workflow**: Emissions compliance forecasting for power plant operations

**Problem Statement**: Power plants must comply with EPA 40 CFR Part 75 emissions monitoring regulations. Predicting emission rates (SO2, NOx, CO2) automates compliance monitoring, addressing the pain point of manual regulatory checks.

**Input Data**: Quarterly CEMS emissions data from EPA CAMPD, including:
- Operational parameters: Gross Load (MWh), Heat Input (mmBtu), Sum of Operating Time
- Categorical features: Primary Fuel Type, Unit Type, Quarter
- Emissions: SO2 Mass, NOx Mass, CO2 Mass, and their rates (lbs/mmBtu or short tons/mmBtu)

**Expected Output**: Predicted emission rates:
- SO2 Rate (lbs/mmBtu)
- NOx Rate (lbs/mmBtu)
- CO2 Rate (short tons/mmBtu)

**Baseline Model**: LinearRegression from scikit-learn for forecasting continuous emission rates.

In [ ]:
# Importing required libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)

print("Libraries imported successfully!")

In [ ]:
# Loading the dataset
try:
    df = pd.read_csv('quarterly-emissions-e59aa51f-1169-445d-918e-fb306c4f282a.csv')
    print(f"Dataset loaded successfully! Shape: {df.shape}")
    print(f"Columns: {list(df.columns)}")
except FileNotFoundError:
    print("Error: Dataset file not found. Please ensure the CSV file is in the working directory.")
    raise

# Display basic data info
print('\nDataset Info:')
print(df.info())
print('\nFirst 5 rows:')
print(df.head())
print('\nDescriptive Statistics:')
print(df.describe())

In [ ]:
# Data Quality Analysis
print("=== DATA QUALITY ANALYSIS ===")
print(f"Total rows: {len(df)}")
print(f"Rows with zero operating time: {len(df[df['Sum of the Operating Time'] == 0])}")
print(f"Missing values per column:")
print(df.isnull().sum())

# Check target variable distributions
targets = ['SO2 Rate (lbs/mmBtu)', 'NOx Rate (lbs/mmBtu)', 'CO2 Rate (short tons/mmBtu)']
print("\n=== TARGET VARIABLE ANALYSIS ===")
for target in targets:
    print(f"\n{target}:")
    print(f"  Mean: {df[target].mean():.4f}")
    print(f"  Std: {df[target].std():.4f}")
    print(f"  Min: {df[target].min():.4f}")
    print(f"  Max: {df[target].max():.4f}")
    print(f"  Missing: {df[target].isnull().sum()}")

# Check categorical variables
print("\n=== CATEGORICAL VARIABLES ===")
categorical_cols = ['Primary Fuel Type', 'Unit Type', 'Quarter']
for col in categorical_cols:
    print(f"\n{col} unique values: {df[col].nunique()}")
    print(df[col].value_counts().head())

In [ ]:
# Improved Data Preprocessing
print("=== DATA PREPROCESSING ===")

# Create a copy for processing
df_processed = df.copy()

# Filter out zero operating time (these are non-operational periods)
print(f"Removing {len(df_processed[df_processed['Sum of the Operating Time'] == 0])} rows with zero operating time")
df_processed = df_processed[df_processed['Sum of the Operating Time'] > 0].copy()

# Remove rows with missing target values
for target in targets:
    before = len(df_processed)
    df_processed = df_processed.dropna(subset=[target])
    after = len(df_processed)
    if before != after:
        print(f"Removed {before - after} rows with missing {target}")

# Feature engineering with better logic
print("\nApplying feature engineering...")

# Only calculate rolling features if we have enough data per facility-unit
facility_unit_counts = df_processed.groupby(['Facility ID', 'Unit ID']).size()
valid_facility_units = facility_unit_counts[facility_unit_counts >= 4].index

# Initialize rolling feature with NaN
df_processed['Rolling_Heat_Input'] = np.nan

# Calculate rolling average only for facility-units with enough data
for (facility_id, unit_id) in valid_facility_units:
    mask = (df_processed['Facility ID'] == facility_id) & (df_processed['Unit ID'] == unit_id)
    df_processed.loc[mask, 'Rolling_Heat_Input'] = df_processed.loc[mask, 'Heat Input (mmBtu)'].rolling(4, min_periods=1).mean()

# Fill remaining NaN values with current heat input
df_processed['Rolling_Heat_Input'].fillna(df_processed['Heat Input (mmBtu)'], inplace=True)

# Load to Heat Ratio with better handling
df_processed['Load_to_Heat_Ratio'] = df_processed['Gross Load (MWh)'] / df_processed['Heat Input (mmBtu)'].replace(0, np.nan)
df_processed['Load_to_Heat_Ratio'].fillna(df_processed['Load_to_Heat_Ratio'].median(), inplace=True)

print(f"Final dataset shape: {df_processed.shape}")
print(f"Remaining missing values: {df_processed.isnull().sum().sum()}")

In [ ]:
# Improved Model Training with Cross-Validation
print("=== MODEL TRAINING AND EVALUATION ===")

# Define features and targets
features = ['Gross Load (MWh)', 'Heat Input (mmBtu)', 'Sum of the Operating Time', 
            'Rolling_Heat_Input', 'Load_to_Heat_Ratio', 'Primary Fuel Type', 'Unit Type', 'Quarter']
targets = ['SO2 Rate (lbs/mmBtu)', 'NOx Rate (lbs/mmBtu)', 'CO2 Rate (short tons/mmBtu)']

# Prepare data
X = df_processed[features].copy()
y = df_processed[targets].copy()

# Handle any remaining missing values in features
numerical_cols = ['Gross Load (MWh)', 'Heat Input (mmBtu)', 'Sum of the Operating Time', 
                  'Rolling_Heat_Input', 'Load_to_Heat_Ratio']
categorical_cols = ['Primary Fuel Type', 'Unit Type', 'Quarter']

for col in numerical_cols:
    X[col].fillna(X[col].median(), inplace=True)

for col in categorical_cols:
    X[col].fillna(X[col].mode()[0] if len(X[col].mode()) > 0 else 'Unknown', inplace=True)

# Create preprocessing pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),
        ('cat', OneHotEncoder(drop='first', handle_unknown='ignore'), categorical_cols)
    ])

# Split data with stratification by quarter to ensure temporal balance
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=X['Quarter'])

print(f"Training set size: {X_train.shape[0]}")
print(f"Test set size: {X_test.shape[0]}")

# Train models for each target
models = {}
results = {}

for target in targets:
    print(f"\nTraining model for {target}...")
    
    # Create pipeline
    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('model', RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42, n_jobs=-1))
    ])
    
    # Cross-validation
    cv_scores = cross_val_score(pipeline, X_train, y_train[target], cv=5, scoring='r2')
    
    # Train final model
    pipeline.fit(X_train, y_train[target])
    
    # Predictions
    y_pred_train = pipeline.predict(X_train)
    y_pred_test = pipeline.predict(X_test)
    
    # Metrics
    train_r2 = r2_score(y_train[target], y_pred_train)
    test_r2 = r2_score(y_test[target], y_pred_test)
    test_mae = mean_absolute_error(y_test[target], y_pred_test)
    test_rmse = np.sqrt(mean_squared_error(y_test[target], y_pred_test))
    
    models[target] = pipeline
    results[target] = {
        'CV_R2_mean': cv_scores.mean(),
        'CV_R2_std': cv_scores.std(),
        'Train_R2': train_r2,
        'Test_R2': test_r2,
        'Test_MAE': test_mae,
        'Test_RMSE': test_rmse
    }
    
    print(f"  CV R²: {cv_scores.mean():.4f} ± {cv_scores.std():.4f}")
    print(f"  Train R²: {train_r2:.4f}")
    print(f"  Test R²: {test_r2:.4f}")
    print(f"  Test MAE: {test_mae:.4f}")
    print(f"  Test RMSE: {test_rmse:.4f}")
    
    if test_r2 < 0:
        print(f"  WARNING: Negative R² indicates model performs worse than baseline!")

In [ ]:
# GridSearchCV Implementation (as specified in project requirements)
print("=== GRIDSEARCHCV HYPERPARAMETER TUNING ===")

from sklearn.model_selection import GridSearchCV

# Define parameter grid for RandomForestRegressor
param_grid = {
    'model__n_estimators': [100, 200, 300],
    'model__max_depth': [10, 15, 20],
    'model__min_samples_split': [5, 10, 15],
    'model__min_samples_leaf': [2, 5, 10]
}

# Perform GridSearchCV for each target
gridsearch_results = {}

for target in targets:
    print(f"\nPerforming GridSearchCV for {target}...")
    
    # Create base pipeline
    base_pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('model', RandomForestRegressor(random_state=42, n_jobs=-1))
    ])
    
    # Perform grid search with cross-validation
    grid_search = GridSearchCV(
        base_pipeline,
        param_grid,
        cv=3,  # 3-fold CV for faster execution
        scoring='r2',
        n_jobs=-1,
        verbose=1
    )
    
    # Fit grid search
    grid_search.fit(X_train, y_train[target])
    
    # Store results
    gridsearch_results[target] = {
        'best_params': grid_search.best_params_,
        'best_score': grid_search.best_score_,
        'best_estimator': grid_search.best_estimator_
    }
    
    # Print results
    print(f"Best parameters for {target}:")
    for param, value in grid_search.best_params_.items():
        print(f"  {param}: {value}")
    print(f"Best cross-validation R² score: {grid_search.best_score_:.4f}")
    
    # Evaluate on test set
    test_predictions = grid_search.best_estimator_.predict(X_test)
    test_r2 = r2_score(y_test[target], test_predictions)
    test_mae = mean_absolute_error(y_test[target], test_predictions)
    
    print(f"Test set performance:")
    print(f"  R² score: {test_r2:.4f}")
    print(f"  MAE: {test_mae:.4f}")
    
    # Compare with our current model
    current_model_r2 = results[target]['Test_R2']
    improvement = test_r2 - current_model_r2
    print(f"  Improvement over current model: {improvement:+.4f} R²")

print("\n=== GRIDSEARCHCV SUMMARY ===")
print("GridSearchCV systematically tested different hyperparameter combinations:")
print("- n_estimators: Number of trees in the forest")
print("- max_depth: Maximum depth of trees")
print("- min_samples_split: Minimum samples required to split a node")
print("- min_samples_leaf: Minimum samples required at a leaf node")
print("\nThis automated tuning process helps find optimal parameters without manual trial-and-error.")

In [ ]:
# Model Evaluation Visualizations
print("=== CREATING VISUALIZATIONS ===")

# Create subplots for predictions vs actual
fig, axes = plt.subplots(1, 3, figsize=(18, 5))
fig.suptitle('Predicted vs Actual Emission Rates', fontsize=16)

for i, target in enumerate(targets):
    y_pred = models[target].predict(X_test)
    
    axes[i].scatter(y_test[target], y_pred, alpha=0.5, s=20)
    
    # Perfect prediction line
    min_val = min(y_test[target].min(), y_pred.min())
    max_val = max(y_test[target].max(), y_pred.max())
    axes[i].plot([min_val, max_val], [min_val, max_val], 'r--', lw=2)
    
    axes[i].set_xlabel(f'Actual {target}')
    axes[i].set_ylabel(f'Predicted {target}')
    axes[i].set_title(f'{target}\nR² = {results[target]["Test_R2"]:.3f}')
    axes[i].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('model_predictions_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

# Feature importance analysis
print("\n=== FEATURE IMPORTANCE ANALYSIS ===")
feature_importance_data = {}

for target in targets:
    # Get feature names after preprocessing
    feature_names = (numerical_cols + 
                    list(models[target].named_steps['preprocessor']
                         .named_transformers_['cat']
                         .get_feature_names_out(categorical_cols)))
    
    # Get feature importances
    importances = models[target].named_steps['model'].feature_importances_
    
    # Create importance dataframe
    importance_df = pd.DataFrame({
        'feature': feature_names,
        'importance': importances
    }).sort_values('importance', ascending=False)
    
    feature_importance_data[target] = importance_df
    
    print(f"\nTop 10 features for {target}:")
    for idx, row in importance_df.head(10).iterrows():
        print(f"  {row['feature']}: {row['importance']:.4f}")

# Plot feature importance
fig, axes = plt.subplots(3, 1, figsize=(12, 15))
fig.suptitle('Feature Importance by Target Variable', fontsize=16)

for i, target in enumerate(targets):
    top_features = feature_importance_data[target].head(10)
    
    axes[i].barh(range(len(top_features)), top_features['importance'])
    axes[i].set_yticks(range(len(top_features)))
    axes[i].set_yticklabels(top_features['feature'])
    axes[i].set_xlabel('Importance')
    axes[i].set_title(f'Top 10 Features for {target}')
    axes[i].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('feature_importance.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Model Diagnostics and Recommendations
print("=== MODEL DIAGNOSTICS AND RECOMMENDATIONS ===")

for target in targets:
    print(f"\n{target} Model Analysis:")
    
    test_r2 = results[target]['Test_R2']
    train_r2 = results[target]['Train_R2']
    cv_r2 = results[target]['CV_R2_mean']
    
    # Overfitting check
    overfitting = train_r2 - test_r2
    if overfitting > 0.1:
        print(f"  ⚠️  OVERFITTING DETECTED: Train R² ({train_r2:.3f}) >> Test R² ({test_r2:.3f})")
        print(f"     Recommendation: Reduce model complexity, increase regularization")
    
    # Performance assessment
    if test_r2 < 0:
        print(f"  🚨 CRITICAL: Model performs worse than baseline (R² = {test_r2:.3f})")
        print(f"     Recommendation: Check data quality, feature engineering, or try different algorithms")
    elif test_r2 < 0.3:
        print(f"  ⚠️  POOR PERFORMANCE: R² = {test_r2:.3f}")
        print(f"     Recommendation: Improve feature engineering, collect more data, or try ensemble methods")
    elif test_r2 < 0.7:
        print(f"  ✅ MODERATE PERFORMANCE: R² = {test_r2:.3f}")
        print(f"     Recommendation: Fine-tune hyperparameters, add domain-specific features")
    else:
        print(f"  🎯 GOOD PERFORMANCE: R² = {test_r2:.3f}")
        print(f"     Recommendation: Model ready for production with monitoring")
    
    # Cross-validation consistency
    cv_std = results[target]['CV_R2_std']
    if cv_std > 0.1:
        print(f"  ⚠️  HIGH VARIANCE: CV std = {cv_std:.3f}")
        print(f"     Recommendation: Model performance is inconsistent across folds")

print("\n=== OVERALL RECOMMENDATIONS ===")
print("1. SO2 Rate model needs significant improvement - investigate data quality")
print("2. Consider time-series specific models for temporal dependencies")
print("3. Add more domain-specific features (weather, maintenance schedules)")
print("4. Implement ensemble methods combining multiple algorithms")
print("5. Set up monitoring for model drift in production")